In [40]:
!pip install pycaret

In [ ]:
#!/bin/bash
curl -L -o ~/Downloads/archive.zip\
https://www.kaggle.com/api/v1/datasets/download/brendan45774/test-file

In [ ]:
!wget https://www.kaggle.com/api/v1/datasets/download/brendan45774/test-file

In [43]:
# Import necessary libraries
import pandas as pd
from pycaret.classification import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Load the Titanic dataset
# Assume the CSV file is saved locally as 'titanic.csv'
df = pd.read_csv('titanic.csv', sep = ',')

# Display the first few rows
print("Dataset Preview:")
df.head()


Dataset Preview:


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [45]:
df.shape[0]

418

In [44]:

# Data Preprocessing
# Drop columns that won't be used for prediction (e.g., PassengerId, Name, Ticket)
df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)

# Convert 'Sex' and 'Embarked' to categorical variables for PyCaret
df['Sex'] = df['Sex'].astype('category')
df['Embarked'] = df['Embarked'].astype('category')

# Handle missing values if necessary (for demonstration, let's fill 'Age' and 'Embarked' with mode/mean)
df['Age'].fillna(df['Age'].mean(), inplace=True)
df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)



In [46]:
from random import seed

seed(2024)
# Train-Test Split
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

train_data.shape, test_data.shape

((334, 8), (84, 8))

In [47]:

# Target variable is 'Survived'
exp_clf = setup(data=train_data, target='Survived', session_id=42, verbose=False)

# Compare models and select the best one based on cross-validation
best_model = compare_models()

# Print the best model selected by AutoML
print("\nBest Model Selected:")
print(best_model)



,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.8860
nb,Naive Bayes,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.0800
dt,Decision Tree Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.0780
ridge,Ridge Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.0830
rf,Random Forest Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.2750
qda,Quadratic Discriminant Analysis,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.1260
ada,Ada Boost Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.1510
gbc,Gradient Boosting Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.1740
et,Extra Trees Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.2050
xgboost,Extreme Gradient Boosting,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.1090


Processing:   0%|          | 0/65 [00:00<?, ?it/s]


Best Model Selected:
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=42, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


In [48]:
# Finalize the model on the entire training dataset
final_model = finalize_model(best_model)

# Predictions and metrics on training data
train_predictions = predict_model(final_model, data=train_data)

# Check the columns in train_predictions to find where the predictions are stored
print("Columns in train_predictions:", train_predictions.columns)

# Make sure to use the correct column for the predicted labels (it may be 'prediction_label' or similar)
predicted_column = 'prediction_label' if 'prediction_label' in train_predictions.columns else 'Label'

# Evaluate metrics on training data
print("\nTraining Data Metrics:")
print(classification_report(train_data['Survived'], train_predictions[predicted_column]))

# Predictions and metrics on test data
test_predictions = predict_model(final_model, data=test_data)
print("\nTest Data Metrics:")
print(classification_report(test_data['Survived'], test_predictions[predicted_column]))

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Logistic Regression,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000


Columns in train_predictions: Index(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked',
       'Survived', 'prediction_label', 'prediction_score'],
      dtype='object')

Training Data Metrics:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       216
           1       1.00      1.00      1.00       118

    accuracy                           1.00       334
   macro avg       1.00      1.00      1.00       334
weighted avg       1.00      1.00      1.00       334



,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Logistic Regression,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000



Test Data Metrics:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        50
           1       1.00      1.00      1.00        34

    accuracy                           1.00        84
   macro avg       1.00      1.00      1.00        84
weighted avg       1.00      1.00      1.00        84



In [49]:
from pycaret.classification import save_model

# Save the finalized model
save_model(final_model, 'titanic_best_model')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(exclude=None,
                                     include=['Pclass', 'Age', 'SibSp', 'Parch',
                                              'Fare'],
                                     transformer=SimpleImputer(add_indicator=False,
                                                               copy=True,
                                                               fill_value=None,
                                                               keep_empty_features=False,
                                                               missing_values=nan,
                                                               strategy='mean'))),
                 ('categorical_imputer',
                  TransformerWrapper(exclude=None, include=['Sex...
                                                               handle_missing='return_nan',
                                       

In [50]:
from pycaret.classification import load_model

# Load the saved model
loaded_model = load_model('titanic_best_model')

# Example: Making predictions with the loaded model on new data
predictions = predict_model(loaded_model, data=test_data)
print(predictions)

Transformation Pipeline and Model Successfully Loaded


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Logistic Regression,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000


     Pclass     Sex   Age  SibSp  Parch        Fare Embarked  Survived  \
321       3    male  25.0      0      0    7.229200        C         0   
324       1  female  39.0      0      0  211.337494        S         1   
388       3    male  21.0      0      0    7.750000        Q         0   
56        3    male  35.0      0      0    7.895800        S         0   
153       3  female  36.0      0      2   12.183300        S         1   
..      ...     ...   ...    ...    ...         ...      ...       ...   
57        3    male  25.0      0      0    7.650000        S         0   
126       3    male  22.0      0      0    7.795800        S         0   
24        1  female  48.0      1      3  262.375000        C         1   
17        3    male  21.0      0      0    7.225000        C         0   
66        3  female  18.0      0      0    7.879200        Q         1   

     prediction_label  prediction_score  
321                 0            0.9764  
324                 1      